Attempt number one at creating a custom neuron class. Basically wrote the class from scratch. Unsuccessful due to not understanding how Nengo calculates J and what its dimensions mean

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist, pdist, squareform

import nengo

n_neurons = 3 # Number of neurons
sqR = 28 # Domain length?
ri = 3
ro = 6

tf = np.zeros((n_neurons, 1))
th = np.ones((n_neurons, 1))
b = 0 * np.ones((n_neurons, 1))
v_reset = 0 + 0.1*(np.random.rand(n_neurons,1)*np.random.rand(n_neurons,1))

nx = sqR*np.random.rand(n_neurons,2)
D = squareform(pdist(nx))
H = np.ones((n_neurons,1))
U = D.dot(H)
print(D)
print(U)
# S = 30*(Ret.D < Ret.ri)- 10*(Ret.D > Ret.ro).*exp(-Ret.D / 10)
# for now randomly set this 3x3 matrix
# S = S - diag(diag(s))    

[[0.         5.99976524 6.18999516]
 [5.99976524 0.         4.54771982]
 [6.18999516 4.54771982 0.        ]]
[[12.1897604 ]
 [10.54748506]
 [10.73771497]]


How to add new neuron type to Nengo:
    https://www.nengo.ai/nengo/examples/usage/rectified-linear.html

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import nengo
from nengo.neurons import settled_firingrate
from nengo.dists import Choice, Uniform
from nengo.params import Parameter, NumberParam, NdarrayParam
from nengo.utils.ensemble import tuning_curves

# Neuron types must subclass `nengo.neurons.NeuronType`
class CustomLIF(nengo.neurons.NeuronType):  
    """Modified version of the leaky integrate-and-fire (LIF) neuron model.

    Parameters
    ----------
    tau_v : float
        Membrane RC time constant, in seconds. Affects how quickly the membrane
        voltage decays to zero in the absence of input (larger = slower decay).
    v_reset : Matrix
        The voltage to reset to after a spike, in millivolts. Noise on activity field.
    th : float
        Voltage threshold for firing. Currently constant, will be changing like voltage later.
    H : Matrix
        
    n_neurons : int
        Total number of neurons in ensemble
    """
    
    probeable = ("spiked", "voltage")
    
    tau_v = NumberParam("tau_v", low=0, low_open=True)
    v_reset = NdarrayParam("v_reset")
    th = NumberParam("th")
    S = NdarrayParam("S")  # adjacency matrix, random for now
#     n_neurons = NumberParam("n_neurons")

    def __init__(
        self, 
        tau_v=2, 
        v_reset=0.1*np.power(np.random.rand(int(n_neurons),1), 2),
        th=30,
        S=squareform(pdist(28*np.random.rand(n_neurons,2))).dot(np.ones((n_neurons,1))),
#         n_neurons=3,
        initial_state=None
    ):
#         super().__init__(initial_state)
#             tau_v=tau_v,
#             tau_ref=tau_ref, 
#             amplitude=amplitude,
            
        
        self.tau_v = tau_v
        self.v_reset = v_reset
        self.th = th 
        self.S = S
        self.n_neurons = n_neurons
        
    def rates(self, x, gain, bias):
        """Estimates steady-state firing rate given gain and bias."""
        J = self.current(x, gain, bias)
        voltage = np.zeros_like(J)
        recovery = np.zeros_like(J)
        return settled_firingrate(
            self.step_math, J, [voltage], settle_time=0.001, sim_time=1.0
        )

        
    def step_math(self, dt, J, spiked, voltage):
        """Implement the differential equation for this neuron type."""
        # Numerical instability occurs for very low inputs.
        # We'll clip them be greater than some value that was chosen by
        # looking at the simulations for many parameter sets.
        # A more principled minimum value would be better.
        J = np.maximum(-30.0, J)
          
        dV = ((-1/self.tau_v) * voltage + self.S)  # plus input signal later
        voltage[:] += dV * dt

        # We check for spikes and reset the voltage here rather than after,
        # which differs from the original implementation by Izhikevich.
        # However, calculating recovery for voltage values greater than
        # threshold can cause the system to blow up, which we want
        # to avoid at all costs.
        
        spiked[:] = (voltage >= self.th) / dt
        voltage[voltage >= self.th] = self.v_reset[voltage >= self.th]
        S[voltage >= self.th, 1] = np.ones((length(fireR), 1))

        

In [3]:
from nengo.builder.operator import Operator

class SimCustomLIF(Operator):
    """Set output to the firing rate of a rectified linear neuron model."""
    
    def __init__(self, output, J, neurons):
        self.output = output  # Output signal of the ensemble
        self.J = J  # Input current from the ensemble
        self.neurons = neurons  # The CustomlIF instance
        
        # Operators must explicitly tell the simulator what signals
        # they read, set, update, and increment
        self.reads = [J]
        self.updates = [output]
        self.sets = []
        self.incs = []
        
    # If we needed additional signals that aren't in one of the
    # reads, updates, sets, or incs lists, we can initialize them
    # by making an `init_signals(self, signals, dt)` method.
    
    def make_step(self, signals, dt, rng):
        """Return a function that the Simulator will execute on each step.
        
        `signals` contains a dictionary mapping each signal to
        an ndarray which can be used in the step function.
        `dt` is the simulator timestep (which we don't use).
        """
        J = signals[self.J]
        output = signals[self.output]
        def step_simcustomlif():
            # Gain and bias are already taken into account here,
            # so we just need to rectify
            output[...] = np.maximum(0, J)
        return step_simsimcustomlif

In [4]:
model = nengo.Network(label='2D Representation', seed=10)
with model:
    nengo.cache.Fingerprint.whitelist(CustomLIF)
    neurons = nengo.Ensemble(3, dimensions=2, neuron_type=CustomLIF())
    sin = nengo.Node(output=np.sin)
    cos = nengo.Node(output=np.cos)
    nengo.Connection(sin, neurons[0])
    nengo.Connection(cos, neurons[1])
    sin_probe = nengo.Probe(sin, 'output')
    cos_probe = nengo.Probe(cos, 'output')
    neurons_probe = nengo.Probe(neurons, 'decoded_output', synapse=0.01)
with nengo.Simulator(model) as sim:
    sim.run(5)
plt.figure()
plt.plot(sim.trange(), sim.data[neurons_probe], label="Decoded output")
plt.plot(sim.trange(), sim.data[sin_probe], 'r', label="Sine")
plt.plot(sim.trange(), sim.data[cos_probe], 'k', label="Cosine")
plt.legend();


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

ValueError: operands could not be broadcast together with shapes (101,1) (3,1) 